### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [1]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

azureopenai_local (AzureOpenAI)
aoai_connection (AzureOpenAI)
default_connection (CognitiveSearch)
cosmodb_connection (Custom)
acs_connection (CognitiveSearch)
azure_form_recognizer_connection (Custom)
acs_connection_test (CognitiveSearch)
aoai_connection_test (AzureOpenAI)
postgres_connection (Custom)


### Test the flow

In [2]:
output = pf.flows.test(
    "../../rag-cosmos-mongo/flow.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
    },
)

print(output)

[2024-03-15 08:35:43,445][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Start to run 15 nodes with concurrency level 16.
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Executing node ConfigLoader. node run id: ccbfba2a-8ec9-4268-b5fd-4d50f2c33357_ConfigLoader_0
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Executing node CheckExistingContext. node run id: ccbfba2a-8ec9-4268-b5fd-4d50f2c33357_CheckExistingContext_0
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Executing node QueryParser. node run id: ccbfba2a-8ec9-4268-b5fd-4d50f2c33357_QueryParser_0
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Executing node FormatConversation. node run id: ccbfba2a-8ec9-4268-b5fd-4d50f2c33357_FormatConversation_0
2024-03-15 08:35:44 -0700   41444 execution.flow     INFO     Executing node Embedding. node run id: ccbfba2a-8ec9-4268-b5fd-4d50f2c33

### Run the flow with the benchmark data file

In [3]:
flow_path = "../../rag-cosmos-mongo/flow.dag.yaml"
data_path = "../../datasets/evalset.csv"

column_mapping = {
    "chat_history": "${data.chat_history}",
    "query": "${data.question}",
}

run_mongo_topk3 = pf.run(flow=flow_path, data=data_path, column_mapping=column_mapping)
pf.stream(run_mongo_topk3)
print(run_mongo_topk3)

: 

In [ ]:
pf.get_details(run_mongo_topk3)

,inputs.chat_history,inputs.query,inputs.line_number,outputs.fetched_docs,outputs.log_params,outputs.reply
0,[],What is the quarterly revenue of Microsoft Clo...,0,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide informa...
1,[],How many Azure Arc customers does Microsoft cu...,1,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide informa...
2,[],Which companies run SAP on Azure in FY23Q1?,2,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide informa...
3,[],What is the annual recurring revenue of GitHub...,3,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide any inf...
4,[],How many monthly active users does Power BI ha...,4,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide informa...
5,[],What is the quarterly revenue of Microsoft Clo...,5,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...","According to the retrieved document, the Micro..."
6,[],How many Azure Arc customers does Microsoft cu...,6,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",The retrieved documents do not provide any inf...
7,[],What are some examples of companies using Azur...,7,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",The retrieved documents do not contain any inf...
8,[],How many developers are using GitHub Copilot a...,8,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",The retrieved documents do not provide any inf...
9,[],What is the growth rate of Azure Machine Learn...,9,"{""retrieved_documents"": [{""[doc[0]"": {""content...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",The retrieved documents do not provide informa...


In [ ]:
ddf = pf.get_details(run_mongo_topk3)
#ddf.to_csv("checkoutput.csv")

## Evaluate 

In [ ]:
eval_flow_path = "../../evaluator/eval_aistudio_score/"
data_path = "../../datasets/evalset.csv"

eval_mongo_run_topk3 = pf.run(
    flow=eval_flow_path,
    run=run_mongo_topk3,
    data=data_path,
    column_mapping={
        "question": "${data.question}",
        "answer": "${data.answer}",
    },
    display_name="eval_mongo_aistudio_score_topk3",
)
pf.stream(eval_mongo_run_topk3)
print(eval_mongo_run_topk3)

In [ ]:
pf.get_details(eval_mongo_run_topk3)

,inputs.question,inputs.answer,inputs.line_number,inputs.context,inputs.log_params,inputs.reply,outputs.coherence,outputs.fluency,outputs.gpt_similarity,outputs.groundedness,outputs.relevance
0,What is the quarterly revenue of Microsoft Clo...,"$25 billion in quarterly revenue, up 24 percen...",0,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,4,5,1,1
1,How many Azure Arc customers does Microsoft cu...,"More than 8,500 Arc customers, more than doubl...",1,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,3,5,5,5
2,Which companies run SAP on Azure in FY23Q1?,"Companies like Chobani, Munich RE, Sodexo, Vol...",2,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,4,5,5,3,1
3,What is the annual recurring revenue of GitHub...,GitHub is now at $1 billion in annual recurrin...,3,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,4,5,5,1,1
4,How many monthly active users does Power BI ha...,Power BI is the market leader in business inte...,4,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,5,5,3,1
5,What is the quarterly revenue of Microsoft Clo...,"$27 billion in quarterly revenue, up 22 percen...",5,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,5,5,1,1
6,How many Azure Arc customers does Microsoft cu...,"More than 12,000 Arc customers, double the num...",6,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,4,5,1,2
7,What are some examples of companies using Azur...,"Over 200 customers, including KPMG and Al Jaze...",7,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,5,5,5,3,1
8,How many developers are using GitHub Copilot a...,More than one million people have used GitHub ...,8,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,4,4,5,1,1
9,What is the growth rate of Azure Machine Learn...,Azure ML revenue alone has increased more than...,9,"[""More than 8,500 Arc customers, more than dou...",{},Over 8500 Arc customers. This is more than dou...,4,4,5,3,1


In [ ]:
pf.get_metrics(eval_mongo_run_topk3)

{'relevance': 1.5,
 'coherence': 4.6,
 'fluency': 4.4,
 'groundedness': 2.2,
 'gpt_similarity': 5.0}

In [ ]:
pf.visualize(eval_mongo_run_topk3)

The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-jy96swvv.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


### Multiple runs 
Using config.yaml file

In [ ]:
import yaml
import promptflow

pf = promptflow.PFClient()

with open("./runs_config.yaml", "r") as file:
    runs_config = yaml.safe_load(file)

# Access the contents of the config.yaml file
print(runs_config)

{'Run1': {'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'topK': 3}, 'Run2': {'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'topK': 5}}


In [ ]:
# function to update param_config systematically 
import os 
def update_param_config(flow_path, config):
    base_dir = os.path.dirname(flow_path)
    param_config_path = os.path.join(base_dir, 'param_config.yaml')

    with open(param_config_path, 'r') as file:
        param_dict = yaml.safe_load(file)

    param_dict.update(config)

    with open(param_config_path, 'w') as file:
        yaml.dump(param_dict, file)


In [ ]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def run_dags(
    flow_path="../../rag-cosmos-mongo/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.question}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        display_name="rag_mongo_run_from_config_" + run_time,
    )


    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.question}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_mongo_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [ ]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
    run_logs[i] = eval_run_from_config

{'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'topK': 3}


{'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'topK': 5}


[2024-03-15 00:27:52,883][promptflow._sdk._submitter.run_submitter][WARNING] - Run eval_aistudio_score_variant_0_20240315_001652_305210 failed when executing in executor.


In [ ]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])

1
The HTML file is generated at 'C:\\Users\\hokhadiv\\AppData\\Local\\Temp\\pf-visualize-detail-u850gjwx.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
2


### Run different flow variants
Now, lets evaluate same flow but on a node variant.

In [ ]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def run_dags(
    flow_path="../../rag-cosmos-mongo/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    variant=None,
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.question}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        variant =variant,
        display_name="rag_mongo_run_from_config_" + run_time,
    )

    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.question}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_mongo_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [ ]:
# i = 0
# run_logs = {}
# for run in runs_config:
#     i += 1
#     print(runs_config[run])
#     copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
#     run_logs[i] = eval_run_from_config

In [ ]:
# for run_id in run_logs:
#     print(run_id)
#     pf.visualize(run_logs[run_id])

In [ ]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(variant= "${DetermineReply.variant_1}", config=runs_config[run])
    run_logs[i] = eval_run_from_config

{'maxTokens': 3500, 'searchType': 'vector', 'dbName': 'earning_calls', 'topK': 3}


[2024-03-15 05:20:30,226][promptflow._sdk._submitter.run_submitter][WARNING] - 1 out of 10 runs failed in batch run. Please check out C:/Users/hokhadiv/.promptflow/.runs/rag_cosmos_mongo_variant_1_20240315_002755_379275 for more details.


In [ ]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])